# Student Distillation Benchmark

In [3]:
import time
import os
import json
import random
import numpy as np
import librosa
import hashlib
import sounddevice as sd
import matplotlib.pyplot as plt
import pickle as pkl
import dill as pickle
import datetime
from torchsummary import summary

import torch
import torch.nn as nn
import torchaudio
from torch.utils.data import DataLoader

from src.resnet import ResNet
from src.train import test
from src.dataset import GoogleSpeechCommandsDataset
import matplotlib.pyplot as plt

/usr/lib/python3/dist-packages/paramiko/transport.py:220: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [4]:
use_cpu = False
is_spiking = False
path = 'models/'
model_name = 'student_resnet_ep60_acc92_sprs0.pt'
data_test_dir = '/home/marcel/Source/Python/DLOptimization/data/speech_commands_test_set_v0.02'
data_cache_dir = '/home/marcel/Source/Python/DLOptimization/cache/data/'

In [7]:
# Load model
torch.set_flush_denormal(True)
device = torch.device("cuda" if not use_cpu and torch.cuda.is_available() else "cpu")
model_path = str(os.path.join(path, model_name))
model = ResNet.from_state_dict(torch.load(model_path), initial_channels=32, stage_channels=[32, 64, 128], stage_stride=[1, 2, 2], spiking=is_spiking)
model.to(device)
criterion = nn.CrossEntropyLoss()

In [8]:
# Load data
test_data = GoogleSpeechCommandsDataset(data_test_dir, data_cache_dir, encoder='mel', augment=0, train=False)
test_data.precache()
test_loader = DataLoader(test_data, batch_size=64 if not is_spiking else 8, shuffle=True)

/home/marcel/Source/Python/Sketchbook/venv/lib/python3.8/site-packages/torchaudio/functional/functional.py:571: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


--- Test Dataset ---
Path:	/home/marcel/Source/Python/DLOptimization/data/speech_commands_test_set_v0.02
Cache:	/home/marcel/Source/Python/DLOptimization/cache/data/speech_commands_test_set_v0.02
Labels:	('up', 'down', 'left', 'right', 'go', 'stop', 'yes', 'no', 'on', 'off')
Length:	4074


In [9]:
summary(model, input_size=(1, 128, 111))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         QuantStub-1          [-1, 1, 128, 111]               0
            Conv2d-2         [-1, 32, 128, 111]             288
       BatchNorm2d-3         [-1, 32, 128, 111]              64
              ReLU-4         [-1, 32, 128, 111]               0
            Conv2d-5         [-1, 32, 128, 111]           9,216
       BatchNorm2d-6         [-1, 32, 128, 111]              64
              ReLU-7         [-1, 32, 128, 111]               0
            Conv2d-8         [-1, 32, 128, 111]           9,216
       BatchNorm2d-9         [-1, 32, 128, 111]              64
      DeQuantStub-10         [-1, 32, 128, 111]               0
      DeQuantStub-11         [-1, 32, 128, 111]               0
        QuantStub-12         [-1, 32, 128, 111]               0
             ReLU-13         [-1, 32, 128, 111]               0
         ResBlock-14         [-1, 32, 1

In [10]:
accuracy, sparsity, significance, gpu_time, cpu_time_mean, cpu_time_std, num_params = test(model, test_loader, criterion, 
                                                                                           cpu_tests=32, return_time=True, log_interval=10, num_steps=5, device=device)


--- Test Results ---
-> Accuracy Validation on CUDA
Test [0/4074 (0%)]	Correct: 44/4074	ACC: 1.08%	Epoch: None
Test [640/4074 (16%)]	Correct: 533/4074	ACC: 13.08%	Epoch: None
Test [1280/4074 (31%)]	Correct: 994/4074	ACC: 24.40%	Epoch: None
Test [1920/4074 (47%)]	Correct: 1482/4074	ACC: 36.38%	Epoch: None
Test [2560/4074 (62%)]	Correct: 1976/4074	ACC: 48.50%	Epoch: None
Test [3200/4074 (78%)]	Correct: 2455/4074	ACC: 60.26%	Epoch: None
Test [3840/4074 (94%)]	Correct: 2926/4074	ACC: 71.82%	Epoch: None
-> Performance Testing on CPU
	-> Model caching...
	-> CPU time testing...
		1. run (0.071713s)
		2. run (0.062405s)
		3. run (0.058186s)
		4. run (0.058019s)
		5. run (0.062156s)
		6. run (0.057166s)
		7. run (0.060472s)
		8. run (0.060378s)
		9. run (0.063813s)
		10. run (0.054924s)
		11. run (0.059308s)
		12. run (0.053054s)
		13. run (0.068546s)
		14. run (0.085825s)
		15. run (0.087715s)
		16. run (0.086878s)
		17. run (0.08653s)
		18. run (0.077075s)
		19. run (0.058235s)
		20. run (0

/home/marcel/Source/Python/EfficientNN/src/train.py:186: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor = torch.tensor(data, dtype=torch.float32)
